In [ ]:
import numpy as np

import cv2
from statistics import median_high
import os
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from keras.layers import Input, Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dense, BatchNormalization, ZeroPadding2D,Activation, AveragePooling2D, concatenate
from keras.models import Model
from keras.optimizers import Adam
from keras.utils import plot_model
from keras.callbacks import ModelCheckpoint, EarlyStopping

import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from sklearn.metrics import confusion_matrix, classification_report, recall_score, f1_score

In [ ]:
path_train = "/kaggle/input/brainmri/Datasest Merged 1/Datasest Merged 1"

In [ ]:
class_names = ['glioma', 'meningioma', 'notumor', 'pituitary']
class_labels = {}
for i, classes in enumerate(class_names, start=0):
  class_labels[classes] = i

In [ ]:
image_size = (130, 130)

In [ ]:
#For Training data

train_data = []
train_labels = []

for folder in os.listdir(path_train):
    print("In folder: {}".format(folder))
    for file in os.listdir(os.path.join(path_train, folder)):
        image_path = os.path.join(path_train, folder, file)
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        image = cv2.resize(image, image_size)  # Resize the image using image_size
        train_data.append(image)
        train_labels.append(class_labels[folder])

train_data = np.array(train_data, dtype='float32')
train_labels = np.array(train_labels, dtype='int32')

train_data = train_data / 255.0 

In [ ]:
train_data[0]

In [ ]:
plt.imshow(train_data[0], cmap='gray')

In [ ]:
# Encode labels
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(train_labels)
labels_encoded = to_categorical(labels_encoded)

# Print the mapping of labels to their encoded values
label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print("Label Mapping:", label_mapping)

In [ ]:
# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(train_data, train_labels, test_size=0.3, random_state=42, shuffle=True)

# Convert to numpy arrays and reshape
X_train = np.array(X_train).reshape(-1, 130, 130, 1)
X_test = np.array(X_test).reshape(-1, 130, 130, 1)

# Define number of classes
num_classes = len(np.unique(train_labels))

print(num_classes)

In [ ]:
# Resize images before converting to numpy arrays
X_train_resized = np.array([cv2.resize(img, (156, 156)) for img in X_train])
X_test_resized = np.array([cv2.resize(img, (156, 156)) for img in X_test])

# Reshape
X_train_resized = X_train_resized.reshape(-1, 156, 156, 1)
X_test_resized = X_test_resized.reshape(-1, 156, 156, 1)


In [ ]:
import numpy as np
print("y_train classes:", np.unique(y_train))
print("y_test classes:", np.unique(y_test))

In [ ]:
import keras
y_train_categorical = keras.utils.to_categorical(y_train, num_classes=4)
y_test_categorical = keras.utils.to_categorical(y_test, num_classes=4)

# Verify the shape (should be (n_samples, 4) for both)
print("y_train_categorical shape:", y_train_categorical.shape)
print("y_test_categorical shape:", y_test_categorical.shape)

# DenseNet-169

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import (Input, Conv2D, BatchNormalization, Activation, concatenate,
                                     AveragePooling2D, GlobalAveragePooling2D, Dense, MaxPooling2D, ZeroPadding2D)
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K

def conv_block(x, growth_rate, name):
    """A building block for a dense block."""
    bn_axis = 3 if K.image_data_format() == 'channels_last' else 1
    x1 = BatchNormalization(axis=bn_axis, epsilon=1.001e-5, name=name + '_0_bn')(x)
    x1 = Activation('relu', name=name + '_0_relu')(x1)
    x1 = Conv2D(4 * growth_rate, 1, use_bias=False, name=name + '_1_conv')(x1)
    x1 = BatchNormalization(axis=bn_axis, epsilon=1.001e-5, name=name + '_1_bn')(x1)
    x1 = Activation('relu', name=name + '_1_relu')(x1)
    x1 = Conv2D(growth_rate, 3, padding='same', use_bias=False, name=name + '_2_conv')(x1)
    x = concatenate([x, x1], axis=bn_axis, name=name + '_concat')
    return x

def transition_block(x, reduction, name):
    """A transition block."""
    bn_axis = 3 if K.image_data_format() == 'channels_last' else 1
    x = BatchNormalization(axis=bn_axis, epsilon=1.001e-5, name=name + '_bn')(x)
    x = Activation('relu', name=name + '_relu')(x)
    x = Conv2D(int(tf.keras.backend.int_shape(x)[bn_axis] * reduction), 1, use_bias=False, name=name + '_conv')(x)
    x = AveragePooling2D(2, strides=2, name=name + '_pool')(x)
    return x

def dense_block(x, blocks, name):
    """A dense block."""
    for i in range(blocks):
        x = conv_block(x, 32, name=name + '_block' + str(i + 1))
    return x

def DenseNet169(input_shape=(130, 130, 1), classes=4):
    """Instantiates the DenseNet-169 architecture."""
    img_input = Input(shape=input_shape)
    
    bn_axis = 3 if K.image_data_format() == 'channels_last' else 1
    
    x = ZeroPadding2D(padding=((3, 3), (3, 3)))(img_input)
    x = Conv2D(64, 7, strides=2, use_bias=False, name='conv1_conv')(x)
    x = BatchNormalization(axis=bn_axis, epsilon=1.001e-5, name='conv1_bn')(x)
    x = Activation('relu', name='conv1_relu')(x)
    x = ZeroPadding2D(padding=((1, 1), (1, 1)))(x)
    x = MaxPooling2D(3, strides=2, name='pool1')(x)
    
    x = dense_block(x, 6, name='conv2')   # 6 conv blocks
    x = transition_block(x, 0.5, name='pool2')
    x = dense_block(x, 12, name='conv3')  # 12 conv blocks
    x = transition_block(x, 0.5, name='pool3')
    x = dense_block(x, 32, name='conv4')  # 24 -> 32 conv blocks
    x = transition_block(x, 0.5, name='pool4')
    x = dense_block(x, 32, name='conv5')  # 16 -> 32 conv blocks
    
    x = BatchNormalization(axis=bn_axis, epsilon=1.001e-5, name='bn')(x)
    x = Activation('relu', name='relu')(x)
    
    x = GlobalAveragePooling2D(name='avg_pool')(x)
    x = Dense(classes, activation='softmax', name='fc')(x)
    
    # Create model
    model = Model(img_input, x, name='densenet169')
    
    return model

# Instantiate the model
DenseNet169_model = DenseNet169(input_shape=(130, 130, 1), classes=4)

# Optional: Display model summary
DenseNet169_model.summary()

In [ ]:
DenseNet169_model.compile(optimizer=Adam(learning_rate=0.00001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# Define the path where the model with the best parameters will be saved
checkpoint_path = "best_model.keras"

# Create a callback that saves the model with the best validation accuracy
checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    monitor='val_accuracy',  # Monitor validation accuracy
    save_best_only=True,
    save_weights_only=False,
    mode='max',  # Save when val_accuracy is maximized
    verbose=1
)

# Create an EarlyStopping callback with patience=5
early_stopping_callback = EarlyStopping(
    monitor='val_accuracy',  # Monitor validation accuracy
    patience=5,  # Stop training if no improvement after 10 epochs
    mode='max',  # Check for maximization of val_accuracy
    verbose=1,
    restore_best_weights=True  # Restore model weights from the epoch with the best validation accuracy
)

# Train the model with the checkpoint and early stopping callbacks
history1 = DenseNet169_model.fit(X_train, y_train,
                    validation_split=0.2,
                    epochs=50,
                    batch_size=32,
                    callbacks=[checkpoint_callback, early_stopping_callback])

# DenseNet-169 With EfficientNet Capabilities

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import (Input, Conv2D, BatchNormalization, Activation, concatenate,
                                     AveragePooling2D, GlobalAveragePooling2D, Dense, MaxPooling2D, ZeroPadding2D)
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K

def conv_block(x, growth_rate, name):
    """A building block for a dense block."""
    bn_axis = 3 if K.image_data_format() == 'channels_last' else 1
    x1 = BatchNormalization(axis=bn_axis, epsilon=1.001e-5, name=name + '_0_bn')(x)
    x1 = Activation('relu', name=name + '_0_relu')(x1)
    x1 = Conv2D(4 * growth_rate, 1, use_bias=False, name=name + '_1_conv')(x1)
    x1 = BatchNormalization(axis=bn_axis, epsilon=1.001e-5, name=name + '_1_bn')(x1)
    x1 = Activation('relu', name=name + '_1_relu')(x1)
    x1 = Conv2D(growth_rate, 3, padding='same', use_bias=False, name=name + '_2_conv')(x1)
    x = concatenate([x, x1], axis=bn_axis, name=name + '_concat')
    return x

def transition_block(x, reduction, name):
    """A transition block."""
    bn_axis = 3 if K.image_data_format() == 'channels_last' else 1
    x = BatchNormalization(axis=bn_axis, epsilon=1.001e-5, name=name + '_bn')(x)
    x = Activation('relu', name=name + '_relu')(x)
    x = Conv2D(int(tf.keras.backend.int_shape(x)[bn_axis] * reduction), 1, use_bias=False, name=name + '_conv')(x)
    x = AveragePooling2D(2, strides=2, name=name + '_pool')(x)
    return x

def dense_block(x, blocks, name):
    """A dense block."""
    for i in range(blocks):
        x = conv_block(x, 38, name=name + '_block' + str(i + 1))  # Growth rate: 32 -> 38
    return x

def DenseNet169_Scaled(input_shape=(156, 156, 1), classes=4):
    """DenseNet-169 with scaled parameters but original layer counts."""
    img_input = Input(shape=input_shape)
    
    bn_axis = 3 if K.image_data_format() == 'channels_last' else 1
    
    x = ZeroPadding2D(padding=((3, 3), (3, 3)))(img_input)
    x = Conv2D(77, 7, strides=2, use_bias=False, name='conv1_conv')(x)  # 64 * 1.2 = 77
    x = BatchNormalization(axis=bn_axis, epsilon=1.001e-5, name='conv1_bn')(x)
    x = Activation('relu', name='conv1_relu')(x)
    x = ZeroPadding2D(padding=((1, 1), (1, 1)))(x)
    x = MaxPooling2D(3, strides=2, name='pool1')(x)
    
    # Original DenseNet-169 block counts
    x = dense_block(x, 6, name='conv2')   # 6 blocks
    x = transition_block(x, 0.5, name='pool2')
    x = dense_block(x, 12, name='conv3')  # 12 blocks
    x = transition_block(x, 0.5, name='pool3')
    x = dense_block(x, 32, name='conv4')  # 32 blocks
    x = transition_block(x, 0.5, name='pool4')
    x = dense_block(x, 32, name='conv5')  # 32 blocks
    
    x = BatchNormalization(axis=bn_axis, epsilon=1.001e-5, name='bn')(x)
    x = Activation('relu', name='relu')(x)
    
    x = GlobalAveragePooling2D(name='avg_pool')(x)
    x = Dense(2458, activation='relu', name='fc_dense')(x)  # 2048 * 1.2 = 2458
    x = tf.keras.layers.Dropout(0.5)(x)
    x = Dense(classes, activation='softmax', name='fc_output')(x)
    
    # Create model
    model = Model(img_input, x, name='densenet169_scaled')
    
    return model

# Instantiate the model
model_DenseNet169_scaled = DenseNet169_Scaled(input_shape=(156, 156, 1), classes=4)

In [ ]:
model_DenseNet169_scaled.compile(optimizer=Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# Define the path where the model with the best parameters will be saved
checkpoint_path = "best_model.keras"

# Create a callback that saves the model with the best validation accuracy
checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    monitor='val_accuracy',  # Monitor validation accuracy
    save_best_only=True,
    save_weights_only=False,
    mode='max',  # Save when val_accuracy is maximized
    verbose=1
)

# Create an EarlyStopping callback with patience=5
early_stopping_callback = EarlyStopping(
    monitor='val_accuracy',  # Monitor validation accuracy
    patience=5,  # Stop training if no improvement after 10 epochs
    mode='max',  # Check for maximization of val_accuracy
    verbose=1,
    restore_best_weights=True  # Restore model weights from the epoch with the best validation accuracy
)

# Train the model with the checkpoint and early stopping callbacks
history2 = model_DenseNet169_scaled.fit(X_train_resized, y_train,
                    validation_split=0.2,
                    epochs=50,
                    batch_size=32,
                    callbacks=[checkpoint_callback, early_stopping_callback])

In [ ]:
plt.figure(figsize=(12, 6))

# Accuracy
plt.subplot(1, 2, 1)
plt.plot(history1.history['accuracy'], label='Train Accuracy')
plt.plot(history1.history['val_accuracy'], label='Val Accuracy', linestyle='--')
plt.title('DenseNet-169 Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(loc='upper left')
plt.grid(True)

# Loss
plt.subplot(1, 2, 2)
plt.plot(history1.history['loss'], label='Train Loss')
plt.plot(history1.history['val_loss'], label='Val Loss', linestyle='--')
plt.title('DenseNet-169 Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(loc='upper right')
plt.grid(True)

plt.show()

In [ ]:
plt.figure(figsize=(12, 6))

# Accuracy
plt.subplot(1, 2, 1)
plt.plot(history2.history['accuracy'], label='Train Accuracy')
plt.plot(history2.history['val_accuracy'], label='Val Accuracy', linestyle='--')
plt.title('DenseNet-169 Scaled Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(loc='upper left')
plt.grid(True)

# Loss
plt.subplot(1, 2, 2)
plt.plot(history2.history['loss'], label='Train Loss')
plt.plot(history2.history['val_loss'], label='Val Loss', linestyle='--')
plt.title('DenseNet-169 Scaled Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(loc='upper right')
plt.grid(True)

plt.show()

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

# Function to calculate specificity and sensitivity for multi-class
def specificity_score_multiclass(conf_matrix):
    specificity_per_class = []
    sensitivity_per_class = []
    
    for i in range(conf_matrix.shape[0]):  # Loop through each class
        tn = np.sum(conf_matrix) - (np.sum(conf_matrix[i, :]) + np.sum(conf_matrix[:, i]) - conf_matrix[i, i])
        fp = np.sum(conf_matrix[:, i]) - conf_matrix[i, i]
        fn = np.sum(conf_matrix[i, :]) - conf_matrix[i, i]
        tp = conf_matrix[i, i]
        
        specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
        sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
        
        specificity_per_class.append(specificity)
        sensitivity_per_class.append(sensitivity)
    
    return specificity_per_class, sensitivity_per_class

# Function to print model metrics and confusion matrix
def print_model_metrics(y_true, y_pred, model_name, class_names):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted')

    # Compute confusion matrix
    conf_matrix = confusion_matrix(y_true, y_pred)
    
    # Compute specificity and sensitivity for each class
    specificity, sensitivity = specificity_score_multiclass(conf_matrix)
    
    # Print metrics
    print(f"\nMetrics for {model_name}:")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}\n")
    
    for i, class_name in enumerate(class_names):
        print(f"{class_name}: Specificity: {specificity[i]:.4f}, Sensitivity: {sensitivity[i]:.4f}")

    # Plot confusion matrix
    plt.figure(figsize=(6, 4))
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', cbar=False, 
                xticklabels=class_names, yticklabels=class_names)
    plt.title(f'Confusion Matrix for {model_name}')
    plt.ylabel('Actual Label')
    plt.xlabel('Predicted Label')
    plt.show()
    
    # Print classification report
    print("\nClassification Report:")
    print(classification_report(y_true, y_pred, target_names=class_names))

# Function to plot one image per class with predicted and true labels
def plot_sample_images_per_class(X, y_true, y_pred, class_names):
    plt.figure(figsize=(12, 8))
    
    shown_classes = set()
    class_indices = {}

    # Find one index per class
    for idx, label in enumerate(y_true):
        if label not in shown_classes:
            class_indices[label] = idx
            shown_classes.add(label)
        if len(shown_classes) == len(class_names):
            break

    for i, (label, idx) in enumerate(class_indices.items()):
        image = X[idx]
        if image.shape[-1] == 1:  # Grayscale
            image = image.squeeze()
        plt.subplot(1, len(class_names), i + 1)
        plt.imshow(image, cmap='gray' if len(image.shape) == 2 else None)
        plt.axis('off')
        plt.title(f"True: {class_names[y_true[idx]]}\nPred: {class_names[y_pred[idx]]}")

    plt.suptitle("Sample Image Per Class with Prediction")
    plt.tight_layout()
    plt.show()

# Define your actual class names
class_names = ["Pituitary", "No Tumor", "Meningioma", "Glioma"]



In [ ]:
# Get predictions from your trained model
y_pred1 = np.argmax(DenseNet169_model.predict(X_test), axis=1)
y_true1 = y_test  # Assuming it's label-encoded already

# Show metrics and plots
print_model_metrics(y_true1, y_pred1, 'DenseNet-169', class_names)
plot_sample_images_per_class(X_test, y_true1, y_pred1, class_names)


In [ ]:
# Get predictions from your trained model
y_pred1 = np.argmax(model_DenseNet169_scaled.predict(X_test_resized), axis=1)
y_true1 = y_test  # Assuming it's label-encoded already

# Show metrics and plots
print_model_metrics(y_true1, y_pred1, 'DenseNet-169 Scaled', class_names)
plot_sample_images_per_class(X_test_resized, y_true1, y_pred1, class_names)
